In [1]:
import os, glob, shutil, json
from pathlib import Path
import SimpleITK as sitk
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import datetime

sys.path.append("/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/src/utils")
from revert_normalisation import get_ct_normalisation_values, revert_normalisation


In [2]:
config = json.load(open("config.json"))
print(config)
# copy files to nnUNet raw data directory
# Load config file

dataset_id = config['dataset_id']
dataset_data_name = config['dataset_data_name']
dataset_data_path = os.path.join(os.environ['nnUNet_raw'], f'Dataset{dataset_id:03d}_{dataset_data_name}') 
fold = 0
dataset_plan = "nnUNetResEncUNetPlans_80G"
input_dir = os.path.join(dataset_data_path, 'imagesVal')



{'dataset_id': 204, 'dataset_data_name': 'synthrad2025_task1_MR_resXL', 'dataset_target_name': 'synthrad2025_task1_CT_resXL', 'data_root': '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Train/Task1', 'preprocessing_CT': 'CT', 'preprocessing_MRI': 'MR', 'preprocessing_mask': 'no mask'}


In [3]:
submission_path = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission"
submission_name = f'task1_{datetime.datetime.now().strftime("%Y%m%d")}_all_data_resXL'
output_dir = os.path.join(submission_path, submission_name, f'DatasetID{dataset_id:03d}_{dataset_data_name}')


In [10]:
os.environ["nnUNet_raw"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/raw"
os.environ["nnUNet_preprocessed"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed"
os.environ["nnUNet_results"] = "/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/results"

data_root = "/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1"
list_data_mri = sorted(glob.glob(os.path.join(data_root, '**','mr.mha'), recursive=True))
list_data_mask = sorted(glob.glob(os.path.join(data_root, '**','mask.mha'), recursive=True))
print("input1 ---", len(list_data_mri), list_data_mri)
print("input2 ---", len(list_data_mask), list_data_mask)

input1 --- 89 ['/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1/AB/1ABA002/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1/AB/1ABA008/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1/AB/1ABA010/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1/AB/1ABA017/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1/AB/1ABA021/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1/AB/1ABA023/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1/AB/1ABA032/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1/AB/1ABA037/mr.mha', '/datasets/work/hb-synthrad2023/source/synthrad2025_data_v2/synthRAD2025_Task1_Val/Task1/AB/1ABA055/mr.mha', '/da

In [11]:
def process_file(data_path, dataset_path, modality_suffix="_0000"):
    filename = data_path.split(os.sep)[-2]
    if not filename.endswith(f'{modality_suffix}.mha'):
        filename = filename + f'{modality_suffix}.mha'
    os.makedirs(os.path.join(dataset_path, 'imagesVal'), exist_ok=True)
    shutil.copyfile(data_path, os.path.join(dataset_path, f'imagesVal/{filename}'))

with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0000"), list_data_mri), total=len(list_data_mri)))

# with ThreadPoolExecutor() as executor:
#     list(tqdm(executor.map(lambda data_path: process_file(data_path, dataset_data_path, "_0001"), list_data_mask), total=len(list_data_mask)))


100%|██████████| 89/89 [00:01<00:00, 56.66it/s]


In [12]:
os.system(f"nnUNetv2_predict -d {dataset_id} -i {input_dir} -o {output_dir} -c 3d_fullres -p {dataset_plan} -tr nnUNetTrainerMRCT -f {fold}")


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

tile :  0.5
There are 89 cases in the source folder
[['/datasets/work/hb-plamri/source/nnUnet_lung/data/nnUNet_raw/Dataset204_synthrad2025_task1_MR_resXL/imagesVal/1ABA002_0000.mha'], ['/datasets/work/hb-plamri/source/nnUnet_lung/data/nnUNet_raw/Dataset204_synthrad2025_task1_MR_resXL/imagesVal/1ABA008_0000.mha'], ['/datasets/work/hb-plamri/source/nnUnet_lung/data/nnUNet_raw/Dataset204_synthrad2025_task1_MR_resXL/imagesVal/1ABA010_0000.mha'], ['/datasets/work/hb-plamri/source/nnUnet_lung/data/nnUNet_raw/Dataset204_synthrad2025_task1_MR_resXL/imagesVal/1ABA017_0000.mha'], ['/datasets/work/hb-

100%|████████████████████████████████████████████| 1/1 [00:02<00:00,  2.16s/it]


sending off prediction to background worker for resampling and export
done with 1ABA002

Predicting 1ABA008:
perform_everything_on_device: True
(69, 329, 368) -0.707 3.713 float16
Reconstruction: MEAN


100%|████████████████████████████████████████████| 2/2 [00:00<00:00,  2.69it/s]


sending off prediction to background worker for resampling and export
done with 1ABA008

Predicting 1ABA010:
perform_everything_on_device: True
(72, 349, 470) -0.7236 3.219 float16
Reconstruction: MEAN


...

100%|████████████████████████████████████████████| 8/8 [00:02<00:00,  2.86it/s]


sending off prediction to background worker for resampling and export
done with 1THB209

Predicting 1THB219:
perform_everything_on_device: True
(120, 500, 553) -0.7705 3.588 float16
Reconstruction: MEAN


100%|████████████████████████████████████████████| 8/8 [00:02<00:00,  3.56it/s]


sending off prediction to background worker for resampling and export
done with 1THB219
(130, 490, 548) -0.717 3.74 float16


0

In [13]:

ct_plan_path = f"/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/preprocessed/Dataset{config['dataset_id'] + 1}_{config['dataset_target_name']}/{dataset_plan}.json"
ct_mean, ct_std = get_ct_normalisation_values(ct_plan_path)
revert_normalisation(output_dir, ct_mean, ct_std, save_path=output_dir + "_revert_norm")
if os.path.exists(output_dir):
    shutil.move(output_dir, output_dir + "_revert_norm/backup_normalised")


CT mean: -726.3590698242188, CT std: 444.69049072265625


100%|██████████| 89/89 [00:46<00:00,  1.93it/s]


# final archive and submission

In [14]:
save_path = os.path.join(submission_path, submission_name, 'Folder')
os.makedirs(save_path, exist_ok=True)

for file in glob.glob(os.path.join(submission_path, submission_name, "*_revert_norm", "*.mha")):
    new_file = os.path.join(save_path, "sct_" + os.path.basename(file))
    shutil.copyfile(file, new_file)
# zip folder
shutil.make_archive(save_path, 'zip', save_path)

'/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/task1_20250718_all_data_resXL/Folder.zip'

In [15]:
print("Copying the line below to the local machine to download the zip file:")
print(f'rsync -avz xin015@virga.hpc.csiro.au:{save_path}.zip ./synthrad2025_submission')

Copying the line below to the local machine to download the zip file:
rsync -avz xin015@virga.hpc.csiro.au:/datasets/work/hb-synthrad2023/work/synthrad2025/bw_workplace/data/nnunet_struct/submission/task1_20250718_all_data_resXL/Folder.zip ./synthrad2025_submission
